In [1]:
import jieba
import re
import nltk

# 读取停用词列表
def get_stopword_list(file):
    with open(file, 'r', encoding='utf-8') as f:    # 
        stopword_list = [word.strip('\n') for word in f.readlines()]
    return stopword_list


def is_chinese(uchar):
    if uchar >= u'\u4e00' and uchar <= u'\u9fa5':
        return True
    else:
        return False

def format_str(content):
    content_str = ''
    for i in content:
        if is_chinese(i):
            content_str = content_str + i
    return content_str

vocab = {}
vocab_id = 0

# 分词 然后清除停用词语
def clean_stopword(str, stopword_list):
    
    # 预处理
    str = str.lower().replace('digit','').replace('http','').replace('\n','')
    str = re.sub(r"\d+", '',str) # delete digit
    # 分词
    clean_word_list = []
    # word_list = jieba.lcut(str,c、ut_all=True)   # 分词后返回一个列表  jieba.cut(）   返回的是一个迭代器
    word_list = jieba.lcut(str) # 精确模式
    # s = ' '.join(str.split())
    # word_list = s.split(' ') # 空格分


    # 去停用词
    for w in word_list:
        if w not in stopword_list:
            if len(w) != 1 :# 过滤单字体 
                clean_word_list.append(w)

    return clean_word_list # 返回一个token_list


stopword_file =r'stopword.txt'
stopword_list = get_stopword_list(stopword_file)

data_file = r'weibo_data_5k.txt'
vocab_file = r'vocab.txt'
output_file = r'output.txt'
freq_file = r'freq.txt'

# 构建词典
with open(data_file, encoding='utf-8',mode='r') as ipf,  open(vocab_file, encoding='utf-8',mode='w') as vbf:
    all_word= []
    for line in ipf:
        # 小写 去数字 去空
        word_list = clean_stopword(line, stopword_list) # 构造vocab 返回当前 一个文档的词频表
        all_word += word_list
    ipf.close()

# nltk 构建词表
cfd = nltk.FreqDist(all_word)

# 保存词频
with open(freq_file, encoding='utf-8',mode='w') as frqf:
    for k,v in sorted(cfd.items(),key=lambda x:x[1], reverse=True)[:2000]:
        frqf.write(k + ' ' + str(v))
        frqf.write('\n')
    frqf.close()

# 选取词频前2k
vocab2k = {}
count = 0
for e in cfd:
    if count < 50:
        vocab2k[e] = count
        count += 1
   
# 保存词表
filename = open(vocab_file,'w') #dict转txt
for k,v in vocab2k.items():
    filename.write(k)
    filename.write('\n')
filename.close()

# 读取词表
with open(data_file, encoding='utf-8',mode='r') as ipf,  open(output_file, encoding='utf-8',mode='w') as opf:
    count = 0
    for line in ipf:
        word_list = clean_stopword(line, stopword_list) 
        temp_dict = {}
        for w in word_list:
            if w in vocab2k: # 前2k的
                w_id  = vocab2k.get(w)
                # print(w_id)
                if w not in temp_dict:
                    temp_dict[w_id] = 1
                else:
                    temp_dict[w_id] += 1         
        dict_str = str(temp_dict)   
        dict_str = str(temp_dict).replace(r"{",'').replace(r"}",'').replace(r": ",':').replace(r", ",' ')
        count = dict_str.count(":")
        opf.write(str(count) + ' ' + dict_str + '\n')
    ipf.close()
    opf.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\cl\AppData\Local\Temp\jieba.cache
Loading model cost 0.452 seconds.
Prefix dict has been built successfully.
